<a href="https://colab.research.google.com/github/Stanislav-Fadeev/ds-education-/blob/main/testovoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=9dd1253ee7d781eeb383e5c173bb3828bea653d3dfdbe19c769f677231a14166
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [3]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [ ]:
# В таблице user actions сохраняются действия покупателей в интернет магазине, структура таблицы
# user_id
#- id пользователя
# product id - id товара
# action
# time
# date
#- действие, просмотр товара, сохранение в корзину, покупка
#- время совершения события
#- дата события
# Для каждого дня рассчитайте, какой процент просмотров товаров завершился покупкой

In [4]:
all_user_ids = np.arange(1,10000)
all_products_ids=np.arange(1,100)
n=10000

In [5]:
user_ids=np.random.choice(all_user_ids,n)
products_ids=np.random.choice(all_products_ids,n)


In [29]:
start_date =pd.to_datetime('2022-01-01')
times=pd.date_range(start_date,periods=n,freq='1min')


In [30]:
user_actions=pd.DataFrame({'user_id':user_ids,
                           'product_id':products_ids,
                           'time':times})


In [120]:
user_actions['action']='view'

In [121]:
to_cart=0.2
print(np.random.binomial(1,to_cart,1))


[0]


In [126]:
def generate_funel_actions(user_id,product_id,time):
  
  to_cart=0.2
  to_purchase=0.4

  df=pd.DataFrame()

  if np.random.binomial(1,to_cart,1)[0]:
    df=pd.DataFrame({ 
                            'user_id':user_id,
                           'product_id':product_id,
                           'time':time+pd.Timedelta(5,unit='s'),
                            'action':'add to cart'},index=[0])
    if np.random.binomial(1,to_purchase,1)[0]:

      df_purchase=pd.DataFrame({
                            'user_id':user_id,
                           'product_id':product_id,
                           'time':time+pd.Timedelta(10,unit='s'),
                            'action':'purchase'},index=[0])
      df=df.append(df_purchase)
  return df

In [123]:
to_cart_df=pd.DataFrame()


In [151]:
to_cart_df

,user_id,product_id,time,action
0,8061,83,2022-01-01 00:08:05,add to cart
0,8061,83,2022-01-01 00:08:10,purchase
0,98,4,2022-01-01 00:21:05,add to cart
0,98,4,2022-01-01 00:21:10,purchase
0,4638,32,2022-01-01 00:27:05,add to cart
...,...,...,...,...
0,7076,34,2022-01-07 22:23:10,purchase
0,28,3,2022-01-07 22:26:05,add to cart
0,28,3,2022-01-07 22:26:10,purchase
0,5799,29,2022-01-07 22:28:05,add to cart


In [167]:
for index,row in user_actions.iterrows():
  users_df=generate_funel_actions(row['user_id'],row['product_id'],row['time'])
  to_cart_df=to_cart_df.append(users_df)


In [168]:
user_actions=user_actions.append(to_cart_df)
user_actions=user_actions.sort_values('time')

In [169]:
user_actions['date']=user_actions.time.dt.date

In [170]:
q = """
SELECT date,
views,
carts,
purchases,
100 * purchases / views as purchase_percentage
FROM (
SELECT date,
count(case when action = 'view'   then 1 else NULL end) as views,
count(case when action = 'add to cart' then 1 else NULL end) as carts,
count(case when action = 'purchase' then 1 else null end) as purchases
FROM user_actions

GROUP BY date);"""

In [171]:
sqldf(q)

,date,views,carts,purchases,purchase_percentage
0,2022-01-01,1440,4280,1740,120
1,2022-01-02,1440,4306,1690,117
2,2022-01-03,1440,4411,1801,125
3,2022-01-04,1440,4446,1823,126
4,2022-01-05,1440,4348,1617,112
5,2022-01-06,1440,4460,1791,124
6,2022-01-07,1360,4266,1736,127
